# 생물정보학 및 실습 1 - Free Project
## LIN28A binding motif predictor 만들기 

### 환경 세팅 하기 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


 Bioconda 환경 설치


In [16]:
!git clone https://github.com/hyeshik/colab-biolab.git
!cd colab-biolab && bash tools/setup.sh
exec(open('colab-biolab/tools/activate_conda.py').read())

Cloning into 'colab-biolab'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 69 (delta 22), reused 54 (delta 13), pack-reused 0
Unpacking objects: 100% (69/69), done.
./
./root/
./root/.vimrc
./root/.condarc
./root/.tmux.conf
./root/.profile
./root/.bashrc.biolab
./root/.bin.priority/
./root/.bin.priority/pip
./root/.bin.priority/pip3
./root/.bin.priority/pip2
--2022-06-06 08:40:23--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76607678 (73M) [application/x-sh]
Saving to: ‘miniconda3.sh’

miniconda3.sh       100%[===================>]  73.06M   123MB/s    in 0.6s    

2022-06-06 08:40:24 (123 MB/s) - ‘miniconda3.sh’ save

In [14]:
!conda install -y bedtools bioawk

In [15]:
%cd ~

/root


In [19]:
%cd /content/drive/MyDrive/binfo1-datapack01/

/content/drive/MyDrive/binfo1-datapack01


### Fig SC3 그리기 

전체 Genome에 대해서 pileup 해주기 

In [ ]:
!samtools mpileup CLIP-35L33G.bam > CLIP-35L33G.pileup
!wc -l CLIP-35L33G.pileup

[mpileup] 1 samples in 1 input files
959697300 CLIP-35L33G.pileup


In [ ]:
!head CLIP-35L33G.pileup

chr1	3056473	N	4	^!G^!G^!G^!G	HGGI
chr1	3056474	N	4	AAAA	HB@I
chr1	3056475	N	4	AAAA	GG?I
chr1	3056476	N	4	TTTT	HGGH
chr1	3056477	N	4	GGGG	HEGI
chr1	3056478	N	4	GGGG	GEEI
chr1	3056479	N	4	AAAA	EA=I
chr1	3056480	N	4	AAAA	@AEH
chr1	3056481	N	4	GGGG	GFFI
chr1	3056482	N	4	TTTT	G=BG


In [ ]:
!tail CLIP-35L33G.pileup

MU069435.1	31102	N	4	GGGG	IDDB
MU069435.1	31103	N	4	GGGG	DEBE
MU069435.1	31104	N	4	GGGG	EDDE
MU069435.1	31105	N	4	TTTT	@><8
MU069435.1	31106	N	4	TTT$T$	DG<;
MU069435.1	31107	N	2	GG	BC
MU069435.1	31108	N	2	GG	B>
MU069435.1	31109	N	2	TT	@3
MU069435.1	31110	N	2	GG	B1
MU069435.1	31111	N	2	G$G$	@6


Filtering with depth (30)

In [ ]:
!awk '$4 > 30 {print $0;}' CLIP-35L33G.pileup > CLIP-35L33G-filt.pileup
!wc -l CLIP-35L33G-filt.pileup

45781030 CLIP-35L33G-filt.pileup


In [ ]:
!tail CLIP-35L33G-filt.pileup

MU069435.1	1559	N	346	>>>>TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT$TT$T$T$TTTTTTTTTTTTTTT$TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT	DIDHGHIIIIIIEEIEIGIIEGGBGGDG:BEBHDDFBIGIEHGIIGIIIHBGIBB>G?DI3IGBEI7GGGGI4GII=G?IIGDIIHIIEHGIHIHGEIHIG=EIHGIIHIGIIIIDIDIIIEIIIIIIGIHIIFI@GGIIIIBIIEIGHH?@IAIGI@HIBHHHFAGIGHGHDIIGIGIIBGIEFHIGGIIIIGHIHIDIIFIHIGGIH@DHGHHDIEDIIEIII8GDHHGHIBGIIHIDIAIGIHIIHEG4II>HD?IH@IFIHGIGDHH;AFGH@EGH:BCHG>IEIDIHIHIIHIIIIIIHHGHIGIIGIGFHGCII=HD@HI?IIIGHGGIHIGIDDH>HFB
MU069435.1	1560	N	341	>>>>TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT$T$T$T$TT$T$TT$T$T$T$TT$TT$TT$TTT$TT$T$T$TT$TT$T$TTT$T$TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT

Match와 Substitution만 남기기 

In [ ]:
import pandas as pd 
import re 
import collections 
import math

def estimate_shannon_entropy(rna_sequence):
    m = len(rna_sequence)
    bases = collections.Counter([tmp_base for tmp_base in rna_sequence])
 
    shannon_entropy_value = 0
    for base in bases:
        # number of residues
        n_i = bases[base]
        # n_i (# residues type i) / M (# residues in column)
        p_i = n_i / float(m)
        entropy_i = p_i * (math.log(p_i, 2))
        shannon_entropy_value += entropy_i
 
    return shannon_entropy_value * -1


def chunk_processing(chunck): # define a function that you will use on chunks
    toremove = re.compile('[<>$*#^]')
    chunck['matches'] = chunck['basereads'].apply(lambda x: toremove.sub('', x))
    chunck['entropy'] = chunck.apply(lambda x: estimate_shannon_entropy(x['matches']), axis=1)
    return chunck


chunk_list = [] # create an empty list to hold chunks
chunksize = 10 ** 6 # set chunk size
for chunk in pd.read_csv('CLIP-35L33G-filt.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'],chunksize=chunksize): # read in csv in chunks of chunksize
    processed_chunk = chunk_processing(chunk) # process the chunks with chunk_processing() function
    chunk_list.append(processed_chunk) # append the chunks to a list
df_concat = pd.concat(chunk_list) # concatenate the list to a dataframe
df_concat.to_csv("CLIP-35L33G-filt.csv", sep='\t')

In [2]:
import pandas as pd 
import re 
import collections 
import math

In [3]:
df_concat = pd.read_csv("entropy_with_exclam.csv", sep='\t')

Filtering with Entropy (0.7) 

In [4]:
filt_entropy = df_concat[df_concat['entropy'] >= 0.7].copy()
# filt_entropy.to_csv("CLIP-35L33G-filt_entropy.csv", sep='\t', index=False, columns=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals', 'entropy'], )
# filt_entropy = pd.read_csv("CLIP-35L33G-filt_entropy.csv", sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals', 'entropy'])
# th_filt_entropy = filt_entropy.sort_values(by='entropy', ascending=False).nlargest(1000, 'entropy')

In [15]:
th_filt_entropy

,Unnamed: 0,chrom,pos,_ref,count,basereads,quals,matches,entropy
37362043,37362043,chr15,55822165,N,51,aaaaaa^RA^@A^NA^FA^IA^NA^NA^CA^MA^MA^JA^QA^@A^...,EHBGHHIDIHIIIEIHHGHHIHGHFBGIGGDIHBFFHGGHIEHIHI...,aaaaaaRA@ANAFAIANANACAMAMAJAQA@AHAKALAYAMAHAEA...,3.308583
15493276,15493276,chr5,134173078,N,46,GGG+1AGGG+2AAGG+2AAG-2NNGGGGGG+3AAAGGGg+1ag+2a...,IFIGBIGIG@@HEBIGHG;74;@4919;;;>G@91>A@;@@>@1;9,GGG+1AGGG+2AAGG+2AAG-2NNGGGGGG+3AAAGGGg+1ag+2a...,3.127228
26710626,26710626,chr11,18277204,N,73,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<...,4IIEIEHBIDG?BB>DIIDIGIIFDDIHHIGGD?IIIDI>EBGH?B...,cccccca-1na-1na-1na+2tca-1na+2tca-1na+2tca+2tc,2.902311
22049311,22049311,chr8,124328383,N,35,<<<<<<<<<c+3gagc+3gagc+3gagc+2gac+2gac+2gac+3g...,GIB1E@IIIDIIEHGG=GIIIDI8IE?IDDDGH5I,c+3gagc+3gagc+3gagc+2gac+2gac+2gac+3gagc+2gac+...,2.886946
38607934,38607934,chr16,17040078,N,49,"^&A^/A^,A^HA^BA^CA^$A^%A^DA^EA^GA^$A^IA^&A^%A^...",GBBBB=GBBEFEF1F=??FFE?4CDGFDBFFFFFFF@CCDEF=4GED96,"&A/A,AHABACAA%ADAEAGAAIA&A%AGACA-A1AAEAGA)A/A'...",2.860853
...,...,...,...,...,...,...,...,...,...
34841614,34841614,chr14,30594522,N,106,<<<<GGGGGGGGG-2NNGCGGGGGGG-2NNGGGG-2NNCGGAGGGG...,EIBIG@IHG;HIGGIIIIEHIF7IHIIGGIBHAIIGCIIIIBIGIF...,GGGGGGGGG-2NNGCGGGGGGG-2NNGGGG-2NNCGGAGGGG-2NN...,2.248762
31286413,31286413,chr12,55111581,N,33,>>>>>AAAAAAAAAAAAAAA^Ga^Ha^Ia^Ca^Ga^Ha^Ia^Da^I...,7HBGHHH8HIG8BIIHBGIBGHI:GHI;IHGHH,AAAAAAAAAAAAAAAGaHaIaCaGaHaIaDaIaHaGaHaIa,2.248675
16974155,16974155,chr6,52743118,N,31,AAA-3NNNA-3NNNA-3NNNAAA-3NNNA-3NNNA-3NNNAAAAAA...,H@GIIIHIDFHIGDGHDHEIBADCGIHGDHF,AAA-3NNNA-3NNNA-3NNNAAA-3NNNA-3NNNA-3NNNAAAAAA...,2.248450
19206603,19206603,chr7,83546998,N,201,AAAG*G**GAG*G*GGGGGG*GGGGGGGGGGG-2NNG-2NNAG-2N...,IGGGE?DHF>II?IDG8IBIID=IHEDI?GG3HHI<>H>HHG@D9I...,AAAGGGAGGGGGGGGGGGGGGGGGGG-2NNG-2NNAG-2NNAAAG-...,2.248366


In [16]:
th_filt_entropy['aux'] = th_filt_entropy[['chrom', 'pos']].groupby('chrom').cumcount()
pos_dict = th_filt_entropy.pivot(index='aux', columns='chrom', values='pos')

In [17]:
pos_dict

chrom,MU069435.1,chr1,chr10,chr11,chr12,chr13,chr14,chr15,chr16,chr17,...,chr3,chr4,chr5,chr6,chr7,chr8,chr9,chrM,chrX,chrY
aux,,,,,,,,,,,,,,,,,,,,,
0,1203.0,24653769.0,92171570.0,18277204.0,116380823.0,67991110.0,26810364.0,55822165.0,17040078.0,40150638.0,...,51149482.0,55390537.0,134173078.0,10807075.0,35430482.0,124328383.0,88331846.0,9819.0,100697211.0,90803211.0
1,NaN,60553089.0,79749392.0,79456842.0,84207076.0,13686357.0,31423710.0,39255799.0,13632335.0,87958235.0,...,125012613.0,123402294.0,108219508.0,3527303.0,6466921.0,34637502.0,59586432.0,NaN,133594165.0,90819813.0
2,NaN,47470075.0,82832050.0,94535999.0,109594553.0,45023297.0,105366021.0,8189045.0,38346102.0,85084367.0,...,152491333.0,51812517.0,52799989.0,108120033.0,79100245.0,111702398.0,45768970.0,NaN,100698399.0,90753132.0
3,NaN,189380992.0,69791817.0,51860971.0,108169978.0,99496836.0,47539776.0,27480134.0,50417485.0,12954908.0,...,157624116.0,16897025.0,45591476.0,98574564.0,97864181.0,34093553.0,107634108.0,NaN,79349986.0,90810959.0
4,NaN,167092388.0,88863683.0,59108486.0,81541114.0,12418153.0,34469819.0,79800657.0,57199646.0,79227787.0,...,31173272.0,79923420.0,110402032.0,108371355.0,120494488.0,43755196.0,57035334.0,NaN,32946078.0,1279545.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,NaN,NaN,NaN,100395262.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,NaN,NaN,NaN,108902948.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,NaN,NaN,NaN,22550390.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
def comp_dropna(df1):
    return {k: v.dropna().to_list() for k,v in df1.items()}

In [19]:
pos_dict_ = comp_dropna(pos_dict)

In [20]:
pos_dict_

{'MU069435.1': [1203.0],
 'chr1': [24653769.0,
  60553089.0,
  47470075.0,
  189380992.0,
  167092388.0,
  93722572.0,
  59731510.0,
  160862788.0,
  153096763.0,
  89024868.0,
  96481211.0,
  59683729.0,
  143627988.0,
  33521480.0,
  180428531.0,
  192427965.0,
  143559799.0,
  60370894.0,
  127993614.0,
  18334786.0,
  15901627.0,
  180420831.0,
  171633401.0,
  36975760.0,
  151660498.0,
  59725568.0,
  180560868.0,
  185143504.0,
  24653365.0,
  86234094.0,
  191086134.0,
  131860410.0,
  4636258.0,
  181032355.0,
  157036439.0,
  38025533.0,
  136455421.0,
  72224188.0,
  59731100.0,
  71668591.0,
  137458698.0,
  118606591.0,
  36849468.0,
  7205343.0,
  10178686.0,
  33511771.0,
  183097600.0,
  118609982.0,
  17774263.0,
  131082125.0,
  133060661.0,
  13801967.0,
  138443394.0,
  165699228.0,
  55055595.0,
  189372938.0,
  87887548.0,
  189386234.0,
  177275050.0,
  57449118.0,
  189390895.0,
  86458176.0,
  59678868.0,
  131842621.0,
  57450181.0,
  191514519.0,
  5747840.0,

sorting 결과에서 앞뒤로 각 10nt씩 21nt단위로 fasta file 만들기 


In [5]:
!pip install pysam

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.1 MB 5.3 MB/s 


In [ ]:
samfile = pysam.AlignmentFile("CLIP-35L33G.bam", "rb")
iter = samfile.fetch("chr1", 24653759, 24653779)

for x in iter:
  if x.reference_start == 24653759:
    print(x.query_sequence[0:21])
    print(x.reference_length)

SRR458758.9033598	16	#0	24577184	40	1S8M76744N12M	*	0	0	AACTTAAAGGTTAACGCTCTT	array('B', [39, 40, 40, 40, 38, 40, 40, 40, 40, 40, 40, 39, 40, 40, 40, 40, 40, 40, 40, 40, 40])	[('MD', '20'), ('NH', 1), ('HI', 1), ('NM', 0), ('SM', 40), ('XQ', 40), ('X2', 0), ('XO', 'UU'), ('XS', '+')]
ATGGTTGTTAGATTTTGGTGA
43
ATGGTTGTTAGATTTTGGTGA
43
ATGGTTGTTAGATTTTGGTGA
43
ATGGTTGTTAGATTTTGGTGA
43
ATGGTTGTTAGATTTTGGTGA
43
ATGGTTGTTAGATTTTGGTGA
43
ATGGTTGTTAGTGATTTTGGT
48
ATGGTTGTTAGTGATTTTGGT
48


In [21]:
import pysam

samfile = pysam.AlignmentFile("CLIP-35L33G.bam", "rb")


with open("xlinked_7.fasta", "a") as xh:
  for poskey, posval in pos_dict_.items():
    print(poskey)
    for v in posval: 
      print(v)
      iter = samfile.fetch(poskey, int(v)-10, int(v)+10)
      for x in iter:
        if (x.reference_start <= int(v)-10) and (x.reference_end >= int(v)+11):
          p = int(v)-10-x.reference_start
          t = x.query_sequence[p:p+21]
          if len(t) != 21:
            continue 
          xh.write(x.query_sequence[p:p+21])
          xh.write("\n")
          break
      ## position - 10 to position + 10 
xh.close()



MU069435.1
1203.0
chr1
24653769.0
60553089.0
47470075.0
189380992.0
167092388.0
93722572.0
59731510.0
160862788.0
153096763.0
89024868.0
96481211.0
59683729.0
143627988.0
33521480.0
180428531.0
192427965.0
143559799.0
60370894.0
127993614.0
18334786.0
15901627.0
180420831.0
171633401.0
36975760.0
151660498.0
59725568.0
180560868.0
185143504.0
24653365.0
86234094.0
191086134.0
131860410.0
4636258.0
181032355.0
157036439.0
38025533.0
136455421.0
72224188.0
59731100.0
71668591.0
137458698.0
118606591.0
36849468.0
7205343.0
10178686.0
33511771.0
183097600.0
118609982.0
17774263.0
131082125.0
133060661.0
13801967.0
138443394.0
165699228.0
55055595.0
189372938.0
87887548.0
189386234.0
177275050.0
57449118.0
189390895.0
86458176.0
59678868.0
131842621.0
57450181.0
191514519.0
5747840.0
150274383.0
160027139.0
34103850.0
141045839.0
78416880.0
78647519.0
39591190.0
118506740.0
74605932.0
87870772.0
59906595.0
74474885.0
71625204.0
160864698.0
36866353.0
161915680.0
171169025.0
150293153.0
1280

In [23]:
!pip install Bio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 270 kB 5.2 MB/s 
     |████████████████████████████████| 2.3 MB 48.7 MB/s 


In [28]:
!wget https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M29/GRCm39.primary_assembly.genome.fa.gz --no-check-certificate

--2022-06-06 06:39:44--  https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M29/GRCm39.primary_assembly.genome.fa.gz
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.197.74
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.197.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 773873008 (738M) [application/octet-stream]
Saving to: ‘GRCm39.primary_assembly.genome.fa.gz’

GRCm39.primary_asse 100%[===================>] 738.02M  19.0MB/s    in 32s     

2022-06-06 06:40:16 (23.4 MB/s) - ‘GRCm39.primary_assembly.genome.fa.gz’ saved [773873008/773873008]



In [59]:
from Bio import SeqIO
import gzip 

with gzip.open("GRCm39.primary_assembly.genome.fa.gz", "rt") as handle:
  seqs = SeqIO.parse(handle, "fasta")
  with open("xlinked_9.fasta", "a") as xh:
    for seq in seqs:
      for poskey, posval in pos_dict_.items():
        print(str(poskey))
        print(type(seq.id), seq.id)
        print(seq.seq[:10])
        if seq.id == poskey:
          for v in posval: 
            print(v)
            print(seq.seq[int(v)-10:int(v)+10])
            xh.write(str(seq.seq[int(v)-10:int(v)+10]))
            xh.write("\n")
            ## position - 10 to position + 10 
  xh.close()


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
NNNNNNNNNN
chr4
<class 'str'> chr7
NNNNNNNNNN
chr5
<class 'str'> chr7
NNNNNNNNNN
chr6
<class 'str'> chr7
NNNNNNNNNN
chr7
<class 'str'> chr7
NNNNNNNNNN
35430482.0
TGTGCGTGTGCACACACACA
6466921.0
GTGGCTTACAGGAAACCCAC
79100245.0
GGGGGATCCCATATCTCCTT
97864181.0
AGCTCCTCACAAAGATTTTT
120494488.0
ATGATTTTAGTGGCCCGAGC
49441101.0
TTATTTTTTTTCTGTTCCTT
44479621.0
TGCCTTTCCCGTGGTTGGTT
90117894.0
ACTTCTCTTCTAAGAAGTCT
45030646.0
AGGCTGGCTAGGCTCCTCCA
140975211.0
AATGTTGCGATTTTTTTTTT
126146445.0
AGCCAGCTGCAGAGCCAAGT
135298830.0
ACTGGTTCTGCTTGTGAAGA
126791752.0
GGGTATCTATCCCCCAACAC
126146498.0
CCACAGTCTGTGTCGGCTGG
113157827.0
AAGTCTTCCAGGGTGAGTAT
30427489.0
CAGGTCATAGCGTGTTATCA
65775201.0
TACATCAAGGGAGATCGTCT
15671079.0
GTCCTCTTCTGGTGTGTCTG
27281032.0
TTTTAATATATTTTTGTGAC
42354721.0
AAATCAATCCTTTTATGTTG
33846971.0
GGACAGTCACTTTGTGGACA
140359011.0
TTGACTCATCAAAGTCTCCA
140791157.0
GTCCCTCACAAGCTGCTGCA
79914811.0
GCACACCATTGGCCACTTTG
26327013.0
GCTATATACCTTCTCCCAAG
12157

WebLogo Result 

hexamer 추출하기 

In [ ]:
from Bio import SeqIO
import gzip 

with gzip.open("GRCm39.primary_assembly.genome.fa.gz", "rt") as handle:
  seqs = SeqIO.parse(handle, "fasta")
  with open("trainset.txt", "a") as xh:
    for seq in seqs:
      for poskey, posval in pos_dict_.items():
        if seq.id == poskey:
          for v in posval: 
            print(v)
            print(seq.seq[int(v)-10:int(v)+10])
            xh.write(str(seq.seq[int(v)-2:int(v)+4]))
            xh.write("\n")
            ## position - 10 to position + 10 
  xh.close()

In [6]:
len(filt_entropy)

487484

In [5]:
fif_filt_entropy = filt_entropy.nlargest(50000, 'entropy').copy()

In [7]:
under_entropy = df_concat[df_concat['entropy'] == 0].sample(50000).copy()

In [8]:
len(under_entropy)

50000

In [9]:
under_entropy.to_csv("CLIP-35L33G-50-under_entropy.csv", sep='\t', index=False, columns=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals', 'entropy'])
# fif_filt_entropy.to_csv("CLIP-35L33G-50-over_entropy.csv", sep='\t', index=False, columns=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals', 'entropy'])

In [27]:
fif_filt_entropy['start'] = fif_filt_entropy['pos'] - 2
fif_filt_entropy['end'] = fif_filt_entropy['pos'] + 4
fif_filt_entropy.to_csv('50-over_entropy.bed', sep='\t', index=False, header=False, columns=['chrom', 'start', 'end', 'entropy']) 

In [28]:
under_entropy['start'] = under_entropy['pos'] - 2
under_entropy['end'] = under_entropy['pos'] + 4
under_entropy.to_csv('50-under_entropy.bed', sep='\t', index=False, header=False, columns=['chrom', 'start', 'end', 'entropy']) 

In [29]:
!bedtools getfasta -fi GRCm39.primary_assembly.genome.fa -bed fifty-over_entropy.bed -fo fifty-over_entropy.fasta
!bedtools getfasta -fi GRCm39.primary_assembly.genome.fa -bed fifty-under_entropy.bed -fo fifty-under_entropy.fasta

In [22]:
!gzip -d GRCm39.primary_assembly.genome.fa.gz

Training Model - CNN 